### ISR Approach 1

In [72]:
import os,sys # ,getpass
sys.path.insert(0,'../../libs')
import pandas as pd 
from utils import load_json, extract_json_string,get_all_files
from oai_ast_utils import OpenAIAssistant_Base
import tqdm,json,time
#from openai import OpenAI
## load API Key
key = load_json('/root/workspace/key/openai_key.json') 
os.environ['OPENAI_API_KEY'] = key['ChatGPT1']['API_KEY']
#os.environ['OPENAI_API_KEY'] = key['ISR']['API_KEY']
#os.environ["OPENAI_API_KEY"] = getpass.getpass(prompt='OpenAI API Token:')

In [73]:
DELETE_FILES = False
UPLOAD_FILES = True
CREATE_AST = False

- Initiate agent

In [74]:
ISRBot = OpenAIAssistant_Base()

- get all files and upload them 
- if already done so, skip

In [75]:
PDF_folder = '/root/workspace/data/DOCs/PDF'
res_folder = '/root/workspace/data/ISR_Results'
all_pdfs = get_all_files(PDF_folder,end_with='.pdf')
file_names = [os.path.basename(f) for f in all_pdfs]

- if it is the fist time do it. Before upload, remove all existing files first 

In [76]:
if UPLOAD_FILES:
    
    if DELETE_FILES:
    # clear all previous files 
        filter_criteria={} #USA_2022.pdf
        sampled_file_ids = ISRBot.FileManager.get_files_info_by(filter_criteria,return_fields=['id'],to_dict=True,to_single_list=True)
        ISRBot.FileManager.delete_files_by_ids(file_ids=sampled_file_ids)
        print('files deleted {}'.format(sampled_file_ids))
    
    ## upload files get files ids
    files_list = []
    for fp in all_pdfs:
        fid,fname = ISRBot.FileManager.upload_file(fp,purpose='assistants')
        files_list.append((fid,fname))
        
    print(ISRBot.FileManager._get_file_info())


['file-AJd4ejfVOhyTKC4CXJjlLQDD', 'file-zrAaLSb2gch6PwhXrbqWFP0x', 'file-0IovNjuAFjxEXjkIqrQ6Vga6'] has been removed from file server.
files deleted ['file-AJd4ejfVOhyTKC4CXJjlLQDD', 'file-zrAaLSb2gch6PwhXrbqWFP0x', 'file-0IovNjuAFjxEXjkIqrQ6Vga6']
                              id    bytes  created_at            filename  \
0  file-USdjzVAJgclWyxxtYc4tIiUO  3466567  1704572287        USA_2022.pdf   
1  file-fRXHBzeSfTYhcu82NfZ5MlL8  3966712  1704572286        UAE_2021.pdf   
2  file-SEKf1SIkmnacDs70Cpfodz5j  2540360  1704572285   Tanzania_2023.pdf   
3  file-8QD58MFCIiObTdKB2U5hh5ON  7302895  1704572283  Indonesia_2023.pdf   
4  file-qWu8glM3tmEn3ymRwbRO3WXo  2814763  1704572281    Belgium_2022.pdf   

  object     purpose     status status_details  
0   file  assistants  processed           None  
1   file  assistants  processed           None  
2   file  assistants  processed           None  
3   file  assistants  processed           None  
4   file  assistants  processed           N

- upload all files and get file name and ids 

- if the first time run it, create a ISR Bot first 

In [77]:
if CREATE_AST: 
    ISRBot.create_assistant(name='ISR Bot',
                        description='This bot is created to help ISR project',
                        instructions="You are an economist in the IMF. Your main task is to review Country Staff reports.")
    print(ISRBot.assistant.name , ISRBot.assistant.id)

- if the bot is already there, load and activate the assistant 

- load all needed files 

In [78]:
loop_files = ISRBot.FileManager.get_files_info_by(filter_criteria={'filename':file_names},return_fields=['id','filename'],to_dict=True)
print(loop_files)

[{'id': 'file-USdjzVAJgclWyxxtYc4tIiUO', 'filename': 'USA_2022.pdf'}, {'id': 'file-fRXHBzeSfTYhcu82NfZ5MlL8', 'filename': 'UAE_2021.pdf'}, {'id': 'file-SEKf1SIkmnacDs70Cpfodz5j', 'filename': 'Tanzania_2023.pdf'}, {'id': 'file-8QD58MFCIiObTdKB2U5hh5ON', 'filename': 'Indonesia_2023.pdf'}, {'id': 'file-qWu8glM3tmEn3ymRwbRO3WXo', 'filename': 'Belgium_2022.pdf'}]


In [90]:
prompt_identify_risks = """
You are an economist in the IMF. Your main task is to review Country Staff reports. Please identify at least 8 major macro critical risks described in the document. Please provide detailed description of each risks. Please return all identified risks in a JSON object.

Output is a JSON object with the following format: {{"risks": [ {{"risk_name": "", "risk_description": "<risk_description1>"}}, {{"risk_name": "", "risk_description": "<risk_description2>"}}, ...... ]}}

Please proceed with the task, keeping in mind the importance of accuracy and clarity in your analysis.

"""

In [80]:
## reload assistant ; sometime there are bugs if keep reusing the same assistant
filter_criteria={'name':['ISR Bot']}
as_id = ISRBot.get_assistants_info_by(filter_criteria,return_fields=['id'])
print(as_id)
## retrieve by id 
assit = ISRBot.get_assistants_by_ids(as_id[0])
print(assit.name, assit.id )
## set the retrieved assistant as current active assistant 
ISRBot._set_active_assistant(assit)

#for fid,fname in files_list:
results = {}
for f_dict in tqdm.tqdm(loop_files):
## update assistant file with specific country file 
    fid,fname = f_dict.get('id'),f_dict.get('filename')
    ass_info_dict = {
                'model':"gpt-4-1106-preview",
                'file_ids':[fid] ### USA file id
                }
    ISRBot.update_current_assistant(**ass_info_dict)
    
    user_input_dict = {"role":"user",
                    "content":prompt_identify_risks
                    }
    msg,status = ISRBot.quick_query(user_input_dict)
    results[fname]={'msg':msg,'status':status}
    time.sleep(2)
    #print(msg)

['asst_4oWbFLPVW77u9qbxFcLSlklb']
ISR Bot asst_4oWbFLPVW77u9qbxFcLSlklb
set ISR Bot as current active assistant.


  0%|          | 0/5 [00:00<?, ?it/s]

Elapsed time: 30.78 s || Status: completed      

 20%|██        | 1/5 [00:38<02:32, 38.09s/it]

Elapsed time: 28.75 s || Status: completed      

...

 60%|██████    | 3/5 [02:18<01:39, 49.70s/it]

Elapsed time: 40.71 s || Status: completed      

 80%|████████  | 4/5 [03:07<00:49, 49.15s/it]

Elapsed time: 34.58 s || Status: completed      

100%|██████████| 5/5 [03:48<00:00, 45.69s/it]


In [81]:
with open(os.path.join(res_folder,'raw_msg.json'), 'w') as file:
    json.dump(results, file, indent=4)

- post process outputs 

In [92]:
## load file 
results = load_json(os.path.join(res_folder,'raw_msg.json'))

In [94]:
excel_out = {}
for k,v in results.items():
    try:
        res_list = json.loads(extract_json_string(results[k]['msg']))['risks']
        res_df = pd.DataFrame(res_list)
        excel_out[k] = res_df
    except:
        
        print('error:{}'.format(k))


In [95]:
excel_out_path = os.path.join(res_folder,'risks.xlsx')
with pd.ExcelWriter(excel_out_path, engine='openpyxl') as writer:
    # Write each DataFrame to a different sheet
    for k,df in excel_out.items():
        df.to_excel(writer, sheet_name=k)
